In [ ]:
%load_ext autoreload
%autoreload 2
from lens_simulation.Lens import Lens, LensType, GratingSettings
from lens_simulation.Medium import Medium, LithiumNiabate

from lens_simulation.structures import SimulationOptions, SimulationParameters
from lens_simulation import utils
import matplotlib.pyplot as plt

import numpy as np


In [ ]:
# create lens
parameters = SimulationParameters(
    A=10000,
    pixel_size=1e-6,
    sim_height=1000e-6,
    sim_width=1000e-6,
    sim_wavelength=488e-9,
    lens_type=LensType.Spherical,
)



In [ ]:

from lens_simulation.Lens import test_escape_path_fits_inside_simulation
from lens_simulation.Simulation import pad_simulation

ep = 0.5
lens_type = LensType.Cylindrical
lens_type = LensType.Spherical

lens = Lens(500e-6, 50e-6, 2.0, LithiumNiabate())
lens.generate_profile(parameters.pixel_size, lens_type, length=200e-6)

print("lens: ", lens.profile.shape)
plt.imshow(lens.profile, cmap="plasma")
plt.title("lens profile")
plt.show()

# aperture
lens.create_custom_aperture(100e-6, 150e-6, type="radial", inverted=False)

# truncation
lens.create_truncation_mask(truncation_height=49e-6, aperture=True)

grating_settings = GratingSettings(
    width = 25e-6,
    distance = 50e-6,
    depth = 20e-6,
    centred = True
)

lens.create_grating_mask(grating_settings, True, True)

lens.apply_masks(grating=True, truncation=True)

# escape path
lens.create_escape_path(parameters, ep)

# pad simulation
lens = pad_simulation(lens, parameters)

# apply all mask
lens.apply_aperture_masks()

fig = utils.plot_apeture_masks(lens)
plt.show()

fig = utils.plot_lens_modifications(lens)
plt.show()



# TODO: add escape path to config


In [ ]:
test_escape_path_fits_inside_simulation(lens, parameters, ep=0.1)
test_escape_path_fits_inside_simulation(lens, parameters, ep=0.5)
test_escape_path_fits_inside_simulation(lens, parameters, ep=1.5)

In [ ]:
from lens_simulation import utils, validation
from lens_simulation.Lens import Lens, generate_lens, LensType
from lens_simulation.Medium import LithiumNiabate
from pprint import pprint

lens_config = utils.load_yaml_config("lens.yaml")

print("before validation")
pprint(lens_config)

lens_config = validation._validate_default_lens_config(lens_config)

print("after validation")
pprint(lens_config)

lens = generate_lens(lens_config, LithiumNiabate())

lens.generate_profile(1e-6)

# external pixel_size, lens_type

import matplotlib.pyplot as plt
plt.imshow(lens.profile)
plt.show()



#### Plot Slices as GIF
Plot the simulation slices as gif



In [ ]:
%load_ext autoreload
%autoreload 2
import os
from lens_simulation import utils
import glob
import imageio
import numpy as np
import matplotlib.pyplot as plt


print(os.getcwd())

path = r"\\ad.monash.edu\home\User028\pcle0002\Documents\repos\lens_simulation\src\lens_simulation\log\rarely-upward-ocelot\safe-hound"

sim_path = os.path.join(path, str(0))
utils.save_propagation_gif(sim_path)

sim_path = os.path.join(path, str(1))
utils.save_propagation_gif(sim_path)


In [ ]:

# stack slices
sim_path = os.path.join(path, str(1))

utils.save_propagation_slices_gif(sim_path)

In [ ]:

print(sim_path)

filenames = sorted(glob.glob(os.path.join(sim_path, "*mm.npy")))

# TODO: might not be possible for very large sims to load full sim, 
# will need to come up with another way to load slices in right format

# sim = None
# print(f"{len(filenames)} slices.")

# vslice = None
# hslice = None


# for i each dim in slice (301)
    # for each j slice (58)
        # get slice column
        # set to vslice arr column
        # vslice[j, i] = slice[i,]

# slice = np.load(filenames[0])
# vslice = np.zeros(shape=(len(filenames), slice.shape[0]))


# for i in range(slice.shape[0]):
#     for j, fname in enumerate(filenames):

#         slice = np.load(fname)
        
#         vslice[j, i] = slice[i, :] 

#         print(f"{i}, {j}")
#         print(f"fname: {os.path.basename(fname)}")
#         print(f"shape: {slice.shape}")
#         print(f"vslice: {vslice.shape})")
#         # print(f"hslice: {hslice.shape})")
    




In [ ]:
import os
import glob


directory = r"//ad.monash.edu/home/User028/pcle0002/Documents/repos/lens_simulation/src/lens_simulation/log/rarely-upward-ocelot"

sim_run_name = os.path.basename(directory)

sim_directories = [path for path in os.listdir(directory) if os.path.isdir(os.path.join(directory, path))] 

print("directory: ", directory)
print("sim_run_name: ", sim_run_name)
print("sim_dir: ", sim_directories)






In [ ]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from lens_simulation import utils
path = r"\\ad.monash.edu\home\User028\pcle0002\Documents\repos\lens_simulation\src\lens_simulation\log\duly-fancy-panda\sweet-mako"

pd.set_option("display.max_rows", None, "display.max_columns", None)

# df_join = load_simulation_data(path)

print(f"path: {path}")
from pprint import pprint

# print("stages: ")
# display(df_stages)

# print("lens: ")
# display(df_lens)


# print("beam: ")
# display(df_beam)


print("join")
# display(df_join)



In [8]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from lens_simulation import utils
# path = r"\\ad.monash.edu\home\User028\pcle0002\Documents\repos\lens_simulation\src\lens_simulation\log\duly-fancy-panda\sweet-mako"
# pd.set_option("display.max_rows", None, "display.max_columns", None)

directory = r"\\ad.monash.edu\home\User028\pcle0002\Documents\repos\lens_simulation\src\lens_simulation\log\barely-ideal-zebra"

df = utils.load_run_simulation_data(directory)

print(f"run: {os.path.basename(directory)}")
display(df.T)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
                                             grating
0  {'width': 1e-05, 'distance': 2e-05, 'depth': 1...
1                                               None
                                             grating
0  {'width': 1e-05, 'distance': 2e-05, 'depth': 1...
1                                               None
                                             grating
0  {'width': 1e-05, 'distance': 2e-05, 'depth': 1...
1                                               None
                                             grating
0  {'width': 1e-05, 'distance': 2e-05, 'depth': 1...
1                                               None
                                             grating
0  {'width': 1e-05, 'distance': 2e-05, 'depth': 1...
1                                               None
                                             grating
0  {'width': 1e-05, 'distance': 2e-05, 'depth': 1...
1          

,0,1,2,3,4,5,6,7,8,9,...,12,13,14,15,16,17,18,19,20,21
lens,lens_1,lens_2,lens_1,lens_2,lens_1,lens_2,lens_1,lens_2,lens_1,lens_2,...,lens_1,lens_2,lens_1,lens_2,lens_1,lens_2,lens_1,lens_2,lens_1,lens_2
output,1.33,1.0,1.33,1.0,1.33,1.0,1.33,1.0,1.33,1.0,...,1.33,1.0,1.33,1.0,1.33,1.0,1.33,1.0,1.33,1.0
start_distance,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
finish_distance,0.002953,0.001677,0.002953,0.001677,0.002953,0.001677,0.002953,0.001677,0.002953,0.001677,...,0.002953,0.001677,0.002953,0.001677,0.002953,0.001677,0.002953,0.001677,0.002953,0.001677
n_slices,NaN,10.0,NaN,10.0,NaN,10.0,NaN,10.0,NaN,10.0,...,NaN,10.0,NaN,10.0,NaN,10.0,NaN,10.0,NaN,10.0
step_size,0.0001,NaN,0.0001,NaN,0.0001,NaN,0.0001,NaN,0.0001,NaN,...,0.0001,NaN,0.0001,NaN,0.0001,NaN,0.0001,NaN,0.0001,NaN
lens_inverted,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
stage,1,2,1,2,1,2,1,2,1,2,...,1,2,1,2,1,2,1,2,1,2
medium,2.348,2.348,2.348,2.348,2.348,2.348,2.348,2.348,2.348,2.348,...,2.348,2.348,2.348,2.348,2.348,2.348,2.348,2.348,2.348,2.348
diameter,0.0003,0.0003,0.0003,0.0003,0.0003,0.0003,0.0003,0.0003,0.0003,0.0003,...,0.0003,0.0003,0.0003,0.0003,0.0003,0.0003,0.0003,0.0003,0.0003,0.0003


In [4]:
display(df[df["exponent"] ==  2.1].T)


,1,3,5,7,9,11,13,15,17,19,...,47,49,51,53,55,57,59,61,63,65
lens,lens_2,lens_2,lens_2,lens_2,lens_2,lens_2,lens_2,lens_2,lens_2,lens_2,...,lens_2,lens_2,lens_2,lens_2,lens_2,lens_2,lens_2,lens_2,lens_2,lens_2
output,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
start_distance,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
finish_distance,0.001677,0.001677,0.001677,0.001677,0.001677,0.001677,0.001677,0.001677,0.001677,0.001677,...,0.001677,0.001677,0.001677,0.001677,0.001677,0.001677,0.001677,0.001677,0.001677,0.001677
n_slices,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,...,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0,10.0
step_size,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
options,"{'use_equivalent_focal_distance': True, 'focal...","{'use_equivalent_focal_distance': True, 'focal...","{'use_equivalent_focal_distance': True, 'focal...","{'use_equivalent_focal_distance': True, 'focal...","{'use_equivalent_focal_distance': True, 'focal...","{'use_equivalent_focal_distance': True, 'focal...","{'use_equivalent_focal_distance': True, 'focal...","{'use_equivalent_focal_distance': True, 'focal...","{'use_equivalent_focal_distance': True, 'focal...","{'use_equivalent_focal_distance': True, 'focal...",...,"{'use_equivalent_focal_distance': True, 'focal...","{'use_equivalent_focal_distance': True, 'focal...","{'use_equivalent_focal_distance': True, 'focal...","{'use_equivalent_focal_distance': True, 'focal...","{'use_equivalent_focal_distance': True, 'focal...","{'use_equivalent_focal_distance': True, 'focal...","{'use_equivalent_focal_distance': True, 'focal...","{'use_equivalent_focal_distance': True, 'focal...","{'use_equivalent_focal_distance': True, 'focal...","{'use_equivalent_focal_distance': True, 'focal..."
lens_inverted,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
stage,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
medium,2.348,2.348,2.348,2.348,2.348,2.348,2.348,2.348,2.348,2.348,...,2.348,2.348,2.348,2.348,2.348,2.348,2.348,2.348,2.348,2.348
